In [1]:
import pandas as pd
import numpy as np

In [87]:
installs = pd.read_csv('data/installs.csv.gzip', dtype=dtypes_dict, parse_dates=['created'])

In [10]:
installs['created'] = pd.to_datetime(installs['created'], format='%Y-%m-%d %H:%M:%S')

In [23]:
installs['created'].head()

0   2019-04-24 06:23:29.495
1   2019-04-24 02:06:01.032
2   2019-04-20 10:15:36.274
3   2019-04-20 21:56:47.151
4   2019-04-20 22:40:41.239
Name: created, dtype: datetime64[ns]

In [40]:
w1_start = '2019-04-18'
w1_end = '2019-04-21'

installs['window_1'] = installs['created'].between(w1_start, w1_end)

In [55]:
installs['created'].loc[installs['created'].between('2019-04-20', w1_end)].sort_values().tail()

95612    2019-04-20 23:59:52.230
95615    2019-04-20 23:59:55.726
75245    2019-04-20 23:59:57.132
11665    2019-04-20 23:59:58.245
459850   2019-04-20 23:59:58.534
Name: created, dtype: datetime64[ns]

In [56]:
w2_start = '2019-04-19'
w2_end = '2019-04-22'

installs['window_2'] = installs['created'].between(w2_start, w2_end)

In [57]:
w3_start = '2019-04-20'
w3_end = '2019-04-23'

installs['window_3'] = installs['created'].between(w3_start, w3_end)

In [58]:
w4_start = '2019-04-21'
w4_end = '2019-04-24'

installs['window_4'] = installs['created'].between(w4_start, w4_end)

In [59]:
w5_start = '2019-04-22'
w5_end = '2019-04-25'

installs['window_5'] = installs['created'].between(w5_start, w5_end)

In [61]:
installs.head()

,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,...,kind,wifi,trans_id,ip_address,device_language,window_1,window_2,window_3,window_4,window_5
0,2019-04-24 06:23:29.495,1,1494519392962156891,4716708407362582887,NaN,False,True,6287817205707153877,NaN,3.739127e+17,...,app_open,NaN,NaN,4243443387795468703,3.3013777759776993e+18,False,False,False,False,True
1,2019-04-24 02:06:01.032,1,1494519392962156891,7143568733100935872,NaN,False,False,6287817205707153877,NaN,7.805539e+18,...,NaN,NaN,NaN,4724288679627032761,3.3013777759776993e+18,False,False,False,False,True
2,2019-04-20 10:15:36.274,1,1494519392962156891,5230323462636548010,NaN,False,True,6287817205707153877,NaN,8.355496e+18,...,app_open,NaN,NaN,8291809486355890410,4.060929664968129e+18,True,True,True,False,False
3,2019-04-20 21:56:47.151,1,1494519392962156891,5097163995161606833,NaN,False,True,6287817205707153877,NaN,2.355772e+18,...,app_open,NaN,NaN,4006811922873399949,3.3013777759776993e+18,True,True,True,False,False
4,2019-04-20 22:40:41.239,1,1494519392962156891,6328027616411983332,NaN,False,False,6287817205707153877,NaN,6.156971e+18,...,NaN,NaN,NaN,3386455054590810771,3.3013777759776993e+18,True,True,True,False,False


In [89]:
print(installs['device_countrycode'].value_counts())
# print(installs['device_countrycode'].isnull().sum())
print(installs['attributed'].value_counts())
# print(installs['attributed'].isnull().sum())
print(installs['implicit'].value_counts())
# print(installs['implicit'].isnull().sum())
print(installs['ref_type'].value_counts())
# print(installs['ref_type'].isnull().sum())

1    481511
Name: device_countrycode, dtype: int64
0    480369
1      1142
Name: attributed, dtype: int64
0    378343
1    103168
Name: implicit, dtype: int64
1    398906
0     82605
Name: ref_type, dtype: int64


In [78]:
installs['wifi'] = installs['wifi'].map({True: 1, False: 0})
installs['wifi'] = installs['wifi'].fillna(-1)

In [79]:
print(installs['wifi'].isnull().sum())
print(installs['wifi'].value_counts())

0
 1.0    235130
-1.0    186682
 0.0     59699
Name: wifi, dtype: int64


In [88]:
installs['device_countrycode'] = installs['device_countrycode'].map({'6287817205707153877': 1})
installs['attributed'] = installs['attributed'].map({True: 1, False: 0})

installs['implicit'] = installs['implicit'].map({True: 1, False: 0})
installs['ref_type'] = installs['ref_type'].map({'1891515180541284343': 1, '1494519392962156891': 0})

In [90]:
installs.head()

,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language
0,2019-04-24 06:23:29.495,1,0,4716708407362582887,NaN,0,1,1,NaN,3.739127e+17,adjust.com,NaN,79837499-2f2a-4605-a663-e322f759424f,app_open,NaN,NaN,4243443387795468703,3.3013777759776993e+18
1,2019-04-24 02:06:01.032,1,0,7143568733100935872,NaN,0,0,1,NaN,7.805539e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,4724288679627032761,3.3013777759776993e+18
2,2019-04-20 10:15:36.274,1,0,5230323462636548010,NaN,0,1,1,NaN,8.355496e+18,adjust.com,NaN,dda99e3c-9c4b-487d-891c-79f0a02cb4a8,app_open,NaN,NaN,8291809486355890410,4.060929664968129e+18
3,2019-04-20 21:56:47.151,1,0,5097163995161606833,NaN,0,1,1,NaN,2.355772e+18,adjust.com,NaN,7010c3ce-0fcf-46c6-9be8-374cc0e20af4,app_open,NaN,NaN,4006811922873399949,3.3013777759776993e+18
4,2019-04-20 22:40:41.239,1,0,6328027616411983332,NaN,0,0,1,NaN,6.156971e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,3386455054590810771,3.3013777759776993e+18


In [ ]:
installs['user_agent_app'] = installs['user_agent']
len_user_agent = len(installs['user_agent'])
for i in range(len_user_agent - 1):
    elem = installs['user_agent'].iloc[i]
    if not pd.isnull(elem):
        elem = elem.split('/')
        installs['user_agent_app'].iloc[i] = elem[0]

print(installs['user_agent_app'].value_counts())

/home/eliana/fiuba/datos7506/envdt/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [91]:
print(installs['device_model'].value_counts().tail())
print('---',installs['device_model'].isnull().sum())
print(installs['event_uuid'].value_counts().tail())
print('---',installs['event_uuid'].isnull().sum())
print(installs['trans_id'].value_counts().tail())
print('---','---',installs['trans_id'].isnull().sum())
print(installs['session_user_agent'].value_counts().tail())
print('---',installs['session_user_agent'].isnull().sum())

2.721349e+18    1
5.330887e+18    1
4.460033e+18    1
2.318100e+18    1
5.476311e+18    1
Name: device_model, dtype: int64
--- 26892
f3af4323-196b-4eb3-a0a8-cf584a0f554d    1
fdab18b9-d58e-4e43-a80d-f8e89794411e    1
ae41e838-b9ed-4d18-812d-22785a5b7c51    1
d03183e1-cdc4-47d0-bd55-4d1024ecee05    1
ae0c5040-2d99-48f4-9790-2e872ce0fa2d    1
Name: event_uuid, dtype: int64
--- 378343
v.2_g.88417_a.75a1118b-5e8f-4763-bc13-e7e0436d5fa0_c.6_t.ua_u.1523fe4922e7bc5b                                              1
1904252007a52d709d7958ae                                                                                                    1
3gggAwCXi56W0j5BXJewd0Kqrr9viNkS_LajCaT3uMLIWk1YAC4zOTQyMDAyLTgzM18yMzM4NV84OTg1MjA1Y2JkN2FhNzQ4YmFmMzAwMDFiY2VmZjcAAAAA    1
1904241819a5cab54029d72d                                                                                                    1
3gggAwCPqm8n9MxAtJ1dRoo9nwVNiNkS_LajCfiAutDEWk1YAC4zOTQyMDAyLTgzM18yMzMxNV8zMTg3NTA1Y2I1ODJlMTQ4YmFmMzAwMDFkMGM